In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os.path
import numpy as  np

In [4]:
from pathlib import Path
input = Path ("/content/drive/MyDrive/dataset/histograms/histogram_veg.npz")
a = np.load(input)
for k in a.files:
    print (k)

output_image
output_yield
output_year
output_locations
output_index


In [5]:
# load data to memory
content   = np.load(input)
images    = content['output_image']
yields    = content['output_yield']
years     = content['output_year']
locations = content['output_locations']
indices   = content['output_index']

#NDVI Calculation

In [6]:
image_NDVI = np.zeros([images.shape[0],32,32])
for i in range(images.shape[0]):
  image_NDVI[i,:,:] = (images[i,4,:,:]-images[i,5,:,:])/(images[i,4,:,:]+images[i,5,:,:]+ np.finfo(float).eps)

#Stability Calculation

In [7]:
import numpy as np
def calculate_ndvi_bar(ndvi_data):
    """Calculate average NDVI for each year for jth polygon."""
    return np.mean(ndvi_data, axis=2)
def calculate_rndvi_and_snndvi(ndvi_data, ndvi_bar):
    """Calculate rNDVI and snNDVI for each pixel."""
    rNDVI = ndvi_data - ndvi_bar[:,:,np.newaxis]
    snNDVI = np.where(ndvi_data >= ndvi_bar[:,:,np.newaxis], 1, 0)
    return rNDVI, snNDVI

def calculate_snndvi_bar(snNDVI_data):
    """Calculate average snNDVI over years for each pixel."""
    return np.mean(snNDVI_data, axis=0)

def calculate_tnndvi(snNDVI_data, snNDVI_bar):
    """Calculate tnNDVI for each pixel."""
    diff = snNDVI_data - snNDVI_bar
    return np.sqrt(np.mean(diff**2, axis=0))

def classify_stability(ndvi_data, ndvi_bar, tnNDVI):
    """Classify each pixel into stability classes."""
    conditions = [
        (ndvi_data >= ndvi_bar[:,:,np.newaxis]) & (tnNDVI < 0.15),
        (ndvi_data <= ndvi_bar[:,:,np.newaxis]) & (tnNDVI > 0.15),
        (tnNDVI >= 0.15)
    ]
    choices = ["SH", "SL", "U"]
    return np.select(conditions, choices)

# Input data (as an example)
# Assuming ndvi_data shape is (years, polygons, pixels)
# For this example, I'm creating random NDVI values for 8 years, 5 polygons, and 100 pixels.
ndvi_data = image_NDVI

# Step 1: Calculate NDVI bar
ndvi_bar = calculate_ndvi_bar(image_NDVI)

# Step 2: Calculate rNDVI and snNDVI
rNDVI, snNDVI = calculate_rndvi_and_snndvi(image_NDVI, ndvi_bar)

# Step 3: Calculate snNDVI bar and tnNDVI
snNDVI_bar = calculate_snndvi_bar(snNDVI)
tnNDVI = calculate_tnndvi(snNDVI, snNDVI_bar)

# Step 4: Classify stability
stability = classify_stability(image_NDVI, ndvi_bar, tnNDVI)

print(stability)

[[['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ...
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['U' 'U' 'U' ... 'U' 'U' 'U']]

 [['SL' 'SL' 'SL' ... 'U' 'U' 'SL']
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ...
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['U' 'U' 'U' ... 'U' 'U' 'U']]

 [['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ...
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['SL' 'SL' 'SL' ... 'SL' 'SL' 'SL']]

 ...

 [['SL' 'SL' 'SL' ... 'U' 'U' 'SL']
  ['SL' 'SL' 'SL' ... 'U' 'U' 'U']
  ['SL' 'SL' 'SL' ... 'U' 'U' 'U']
  ...
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['U' 'U' 'U' ... 'U' 'U' 'U']
  ['U' 'U' 'U' ... 'U' 'U' 'U']]

 [['SL' 'SL' 'SL' ... 'U' 'U' 'SL']
  ['SL' 'SL' 'SL' ... 'U' 'U' 'U']
  ['SL' 'SL' 'SL' ... 'U' 'U' 'U']
  ...
  ['SL' 'SL' '